In [1]:
import os

In [2]:
%pwd

'c:\\Users\\rahul\\Desktop\\Project\\Text-Summarizer\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\rahul\\Desktop\\Project\\Text-Summarizer'

# Entity: Return type

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_dir: Path
    unzip_dir: Path

# Config Manager

In [6]:
from textsummarization.constants import *
from textsummarization.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self,
                 config_path = CONFIG_FILE_PATH,
                 params_path = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_url = config.source_url,
            local_dir = config.local_dir,
            unzip_dir = config.unzip_dir
            )
        
        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from textsummarization.logging import logger
from textsummarization.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_dir):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_dir
            )
            logger.info(f"Downloaded file {self.config.local_dir} to {self.config.local_dir}")
        else:
            logger.info(f"File {self.config.local_dir} already exists of size {get_size(Path(self.config.local_dir))}")

    def extract_zip_file(self):
        """
        zip_file_path: Path to the zip file
        Extracts all files from the zip file to the unzip_path directory.
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok = True)
        with zipfile.ZipFile(self.config.local_dir, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-08-27 15:58:28,077: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-27 15:58:28,078: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-27 15:58:28,079: INFO: common: Created directory at: artifacts]
[2024-08-27 15:58:28,079: INFO: common: Created directory at: artifacts/data_ingestion]
[2024-08-27 15:58:28,978: INFO: 3066304674: Downloaded file artifacts/data_ingestion/data.zip to artifacts/data_ingestion/data.zip]
